In [24]:
#Hacer datos legibles

import csv
import numpy
import codecs

matrix=[]
f  = open('datos_nuevos.csv', "rt",encoding="utf8")
read = csv.reader(f)
for row in read :
    matrix.append(row)
matrix[0][0]="DIA"
for i in range(1,len(matrix)):
    for j in range(0,3):
        if j==0:
            matrix[i][j]="D"+str(matrix[i][j])
        elif j==1:
            matrix[i][j]="M"+str(matrix[i][j])
        else:
            matrix[i][j]="Y"+str(matrix[i][j])
with open('datos_legibles.csv', 'w',newline="") as csvFile:
    writer = csv.writer(csvFile)
    COLUMNAS=["PRODUCTO", "DIA", "MES", "YEAR", "EURO", "DOLAR", "COBRE", "DESEMPLEO", "EXPMES", "EXP2MES", "VALOR"]
    writer.writerow(COLUMNAS)
    for j in range(1, 304):
        nombre=matrix[0][8+j]
        for i in range (1,len(matrix)):
            dia=matrix[i][0]
            mes=matrix[i][1]
            year=matrix[i][2]
            euro=matrix[i][3]
            dolar=matrix[i][4]
            cobre=matrix[i][5]
            desempleo=matrix[i][6]
            expmes=matrix[i][7]
            exp2mes=matrix[i][8]
            valor=matrix[i][j+8]
            fila=[nombre, dia, mes, year, euro, dolar, cobre, desempleo, expmes, exp2mes, valor]
            writer.writerow(fila)
csvFile.close()

In [25]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
# Read in data and display first 5 rows
features = pd.read_csv('datos_legibles.csv', encoding = "ISO-8859-1", engine='python')
features.head(5)

,PRODUCTO,DIA,MES,YEAR,EURO,DOLAR,COBRE,DESEMPLEO,EXPMES,EXP2MES,VALOR
0,ARROZ,D8,M1,Y2018,724.745,605.62,3.224,6.5,0.1,0.1,101.03
1,ARROZ,D8,M2,Y2018,738.680,603.15,3.082,6.7,0.1,0.4,100.90
2,ARROZ,D8,M3,Y2018,746.365,606.21,3.079,6.9,0.3,0.2,100.18
3,ARROZ,D8,M4,Y2018,744.465,604.25,3.077,6.7,0.2,0.2,101.02
4,ARROZ,D8,M5,Y2018,752.685,634.40,3.059,7.0,0.2,0.2,100.33


In [26]:
print('The shape of our features is:', features.shape)

The shape of our features is: (4545, 11)


In [27]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)
# Display the first 5 rows of the last 12 columns
features.head(5)

,EURO,DOLAR,COBRE,DESEMPLEO,EXPMES,EXP2MES,VALOR,PRODUCTO_ACCESORIOSDEILUMINACION,PRODUCTO_ACCESORIOSDEVESTIR,PRODUCTO_ACCESORIOSELECTRICOS,...,MES_M2,MES_M3,MES_M4,MES_M5,MES_M6,MES_M7,MES_M8,MES_M9,YEAR_Y2018,YEAR_Y2019
0,724.745,605.62,3.224,6.5,0.1,0.1,101.03,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,738.680,603.15,3.082,6.7,0.1,0.4,100.90,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,746.365,606.21,3.079,6.9,0.3,0.2,100.18,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,744.465,604.25,3.077,6.7,0.2,0.2,101.02,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,752.685,634.40,3.059,7.0,0.2,0.2,100.33,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [28]:
predecir = features.loc[(features['YEAR_Y2019'] == 1) & (features['MES_M1'] == 0)]
print(predecir.shape)
features = features.loc[(features['YEAR_Y2019']==0) | (features['MES_M1'] == 1)]

(606, 325)


In [29]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(features["VALOR"])
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop("VALOR", axis = 1)
predecir= predecir.drop("VALOR", axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [30]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size =0, random_state = 42)

In [31]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (3939, 324)
Training Labels Shape: (3939,)
Testing Features Shape: (0, 324)
Testing Labels Shape: (0,)


In [32]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [33]:
febrero_2019=predecir.loc[(predecir['MES_M2'] == 1) & (predecir['YEAR_Y2019']==1)]
marzo_2019=predecir.loc[(predecir['MES_M3'] == 1) & (predecir['YEAR_Y2019']==1)]

In [34]:
pondera=[]
ff  = open('pond.csv', "rt")
r = csv.reader(ff)
for row in r :
    pondera.append(row)
pondera.pop(0)

['prod', 'pond']

In [35]:
IPC_FEBRERO=0
for producto in pondera:
    glosa=producto[0]
    pond=producto[1]
    for i in range(0,303):
        auxx=febrero_2019.iloc[i][glosa].astype(float)
        if auxx==1:
            pronostico=rf.predict([febrero_2019.iloc[i]]).astype(float)
            IPC_FEBRERO=IPC_FEBRERO+pronostico*float(pond)/100                                           
print(IPC_FEBRERO)


[100.1545142]
